Давайте попробуем написать многослойный перцептрон с нуля, пользуясь знаниями, полученными на лекции: реализуем его в numpy. 

In [ ]:
import numpy as np

Мы с вами знаем, что основная часть нейронной сети - это нейрон, или линейная регрессия, пропущенная через функцию активации. Давайте напишем класс для нашего будущего нейрона, а заодно и функцию активации, через которую будет проходить сигнал:

In [ ]:
def sigmoid(x):
    # Наша функция активации: f(x) = 1 / (1 + e^(-x))
    # your code here

class Neuron:
    def __init__(self, weights, bias):
        self.weights = weights
        self.bias = bias

    def feedforward(self, inputs):
        # Умножаем входы на веса, прибавляем порог, затем используем функцию активации

weights = np.array([0, 1]) # w1 = 0, w2 = 1
bias = 4                   # b = 4
n = Neuron(weights, bias)

x = np.array([2, 3])       # x1 = 2, x2 = 3
print(n.feedforward(x))    # 0.9990889488055994

Чтобы создать полноценную нейронную сеть, нужно объединить нейроны в слой. Давайте реализуем следующую архитектуру:

<img src="https://media.proglib.io/posts/2020/10/02/de81e6549b3e3c3bc1e3fdc78fe59f9c.png" />

У этой сети два входа, скрытый слой с двумя нейронами ($h_1$ и $h_2$) и выходной слой с одним нейроном ($o_1$). Обратите внимание, что входы для $o_1$ – это выходы из $h_1$ и $h_2$. Именно это создает из нейронов сеть.

Давайте реализуем прямую связь для нашей нейронной сети.

In [ ]:
class OurNeuralNetwork:
    '''
    Нейронная сеть с:
    - 2 входами
    - скрытым слоем с 2 нейронами (h1, h2)
    - выходным слоем с 1 нейроном (o1)
    Все нейроны имеют одинаковые веса и пороги:
    - w = [0, 1]
    - b = 0
    '''
    def __init__(self):
        weights = np.array([0, 1])
        bias = 0

        # Используем класс Neuron, написанный выше
        self.h1 = Neuron(weights, bias)
        self.h2 = Neuron(weights, bias)
        self.o1 = Neuron(weights, bias)

    def feedforward(self, x):
        out_h1 = self.h1.feedforward(x)
        out_h2 = self.h2.feedforward(x)

        # Входы для o1 - это выходы h1 и h2
        out_o1 = self.o1.feedforward(np.array([out_h1, out_h2]))

        return out_o1

network = OurNeuralNetwork()
x = np.array([2, 3])
print(network.feedforward(x)) # 0.7216325609518421

Допустим, у нас есть следующие измерения:

| Имя | Вес (в фунтах) | Рост (в дюймах) | Пол |
|---|---|---|---|
| Алиса | 133 (54.4 кг) | 65 (165,1 см) | Ж |
| Боб | 160 (65,44 кг) | 72 (183 см) | М |
| Чарли | 152 (62.2 кг) | 70 (178 см) | М |
| Диана | 120 (49 кг) | 60 (152 см) | Ж |


Давайте обучим нашу нейронную сеть предсказывать пол человека по его росту и весу.

Мы будем представлять мужской пол как 0, женский – как 1, а также сдвинем данные, чтобы их было проще использовать:

| Имя | Вес - 135 | Рост - 66 | Пол |
|---|---|---|---|
| Алиса | -2 | -1 | 1 |
| Боб | 25 | 6 | 0 |
| Чарли | 17 | 4 | 0 |
| Диана | -15 | -6 | 1 |

Сделаем сдвиг, чтобы было проще считать (обычно сдвигают на среднее арифметическое, но здесь циферки просто подобраны, чтобы потом было легче). 

Прежде чем обучать нашу нейронную сеть, нам нужно как-то измерить, насколько "хорошо" она работает, чтобы она смогла работать "лучше". Это измерение и есть потери (loss).

Мы используем для расчета потерь среднюю квадратичную ошибку (mean squared error, MSE):

$$MSE = \frac{1} {n} \sum_{i=1}^{n} (y_{true} - y_{pred})^2 $$

In [ ]:
def mse_loss(y_true, y_pred):
    # y_true и y_pred - массивы numpy одинаковой длины
    # your code here

y_true = np.array([1, 0, 0, 1])
y_pred = np.array([0, 0, 0, 0])

print(mse_loss(y_true, y_pred)) # 0.5

Теперь у нас есть четкая цель: минимизировать потери нейронной сети. Мы знаем, что можем изменять веса и пороги нейронов, чтобы изменить ее предсказания, но как нам делать это таким образом, чтобы минимизировать потери?

Будем рассматривать функцию потерь как функцию от весов и порогов. Давайте отметим все веса и пороги нашей нейронной сети:

<img src="https://media.proglib.io/posts/2020/10/03/b3ae8de6555967d30fd4092654358e18.png" />

Теперь мы можем записать функцию потерь как функцию от нескольких переменных:

$$L(w_1, w_2, w_3, w_4, w_5, w_6, b_1, b_2, b_3)$$

Предположим, мы хотим отрегулировать $w_1$. Как изменится значение потери L при изменении $w_1$? На этот вопрос может ответить частная производная $\frac {\partial L} {\partial w_1}$. Как мы ее рассчитаем?

Прежде всего, давайте перепишем эту частную производную через $\frac {\partial y_{pred}} {\partial w_1}$, воспользовавшись цепным правилом. 

$\frac{\partial L} {\partial w_1} = \frac{\partial L} {\partial y_{pred}} * \frac{\partial y_{pred}}{\partial w_1}$

Мы можем рассчитать $\frac{\partial L}{\partial y_{pred}}$, поскольку $y_{true} = 1$, а значит, $L = (1 - y_{pred})^2$:

$\frac{\partial L} {\partial y_{pred}} = \frac{\partial (1 - y_{pred})^2} {\partial y_{pred}} = -2(1 - y_{pred})$

Теперь давайте решим, что делать с $\frac{\partial y_{pred}}{\partial w_1}$. Обозначая выходы нейронов, как прежде, $h_1$, $h_2$ и $o_1$, получаем:

$y_{pred} = o_1 = f(w_5 h_1 + w_6 h_2 + b_3)$

Вспомните, что f() – это наша функция активации, сигмоида. Поскольку $w_1$ влияет только на $h_1$ (но не на $h_2$), мы можем снова использовать цепное правило и записать:

$\frac{\partial y_{pred}}{\partial w_1} = \frac{\partial y_{pred}}{\partial h_1} * \frac{\partial h_1}{\partial w_1}$

$\frac{\partial y_{pred}}{\partial h_1} = w_5 * f'(w_5 h_1 + w_6 h_2 + b_3)$
 
Мы можем сделать то же самое для $\frac{\partial h_1}{\partial w_1}$, снова применяя цепное правило:

$h_1 = f(w_1 x_1 + w_2 x_2 + b_1)$

$\frac{\partial h_1}{\partial w_1} = x_1 * f'(w_1 x_1 + w_2 x_2 + b_1)$
 
 
В этой формуле $x_1$ – это вес, а $x_2$ – рост. Вот уже второй раз мы встречаем $f'(x)$ – производную сигмоидной функции! Давайте вычислим ее:

$f(x) = \frac{1}{1 + e^{-x}}$

$f`(x) = \frac{e^{-x}}{(1 + e^{-x})^2} = f(x) * (1 - f(x))$
 
 
Мы используем эту красивую форму для $f'(x)$ позже. На этом мы закончили! Мы сумели разложить $\frac {\partial L} {\partial w_1}$ на несколько частей, которые мы можем рассчитать:

$\frac{\partial L} {\partial w_1} = \frac{\partial L} {\partial y_{pred}} * \frac{\partial y_{pred}}{\partial h_1} * \frac{\partial h_1}{\partial w_1}$

Такой метод расчета частных производных "от конца к началу" называется методом обратного распространения (backpropagation).

Будем считать, что наш набор данных пока состоит из одной Алисы (стохастический градиентный спуск!):

| Имя | Вес - 141 | Рост - 67 | Пол |
|---|---|---|---|
| Алиса | -2 | -1 | 1 |

Давайте инициализируем все веса как 1, а все пороги как 0. Если мы выполним прямой проход по нейронной сети, то получим:

$h_1 = f(w_1 x_1 + w_2 x_2 + b_1) = f(-2 - 1 + 0) = 0.0474$
$h_2 = f(w_3 x_1 + w_4 x_2 + b_2) = 0.0474$
$o_1 = f(w_5 h_1 + w_6 h_2 + b_3) = f(0.0474 + 0.0474 + 0) = 0.524$

Наша сеть выдает $y_{pred} = 0.524$, что находится примерно на полпути между мужским полом (0) и женским (1). Давайте рассчитаем $\frac{\partial L} {\partial w_1}$:

    # посчитайте это самостоятельно!

Должно получиться 0.0214.

Результат говорит нам, что при увеличении $w_1$ функция ошибки чуть-чуть повышается. 

Давайте используем алгоритм оптимизации под названием стохастический градиентный спуск (stochastic gradient descent), который определит, как мы будем изменять наши веса и пороги для минимизации потерь. Фактически, он заключается в следующей формуле обновления:

$w_1 \leftarrow w_1 - \eta \frac{\partial L} {\partial w_1}$

$\eta$ - гиперпараметр, называемый скоростью обучения. 

Итого, все, что нам нужно теперь делать - вычитать по описанной выше формуле. 

Процесс обучения сети будет выглядеть примерно так:

1. Выбираем одно наблюдение из набора данных. Именно то, что мы работаем только с одним наблюдением, делает наш градиентный спуск стохастическим.
2. Считаем все частные производные функции потерь по всем весам и порогам ($\frac{\partial L} {\partial w_1}$, $\frac{\partial L} {\partial w_2}$ и т.д.)
3. Используем формулу обновления, чтобы обновить значения каждого веса и порога.
4. Снова переходим к шагу 1.

Итак, соберем финальный код для всей нашей сети:

In [ ]:
def sigmoid(x):
    # Наша функция активации: f(x) = 1 / (1 + e^(-x))
    # your code here

def deriv_sigmoid(x):
    # Производная сигмоиды: f'(x) = f(x) * (1 - f(x))
    # your code here

def mse_loss(y_true, y_pred):
    # y_true и y_pred - массивы numpy одинаковой длины
    # your code here

class OurNeuralNetwork:
    '''
    Нейронная сеть с:
    - 2 входами
    - скрытым слоем с 2 нейронами (h1, h2)
    - выходной слой с 1 нейроном (o1)
    '''
    def __init__(self):
        # Веса
        self.w1 = np.random.normal()
        self.w2 = np.random.normal()
        self.w3 = np.random.normal()
        self.w4 = np.random.normal()
        self.w5 = np.random.normal()
        self.w6 = np.random.normal()

        # Пороги
        self.b1 = np.random.normal()
        self.b2 = np.random.normal()
        self.b3 = np.random.normal()

    def feedforward(self, x):
        # x - массив numpy с двумя элементами
        h1 = sigmoid(self.w1 * x[0] + self.w2 * x[1] + self.b1)
        h2 = sigmoid(self.w3 * x[0] + self.w4 * x[1] + self.b2)
        o1 = sigmoid(self.w5 * h1 + self.w6 * h2 + self.b3)
        return o1

    def train(self, data, all_y_trues):
        '''
        - data - массив numpy (n x 2) numpy, n = к-во наблюдений в наборе. 
        - all_y_trues - массив numpy с n элементами.
          Элементы all_y_trues соответствуют наблюдениям в data.
        '''
        learn_rate = 0.1
        epochs = 1000 # сколько раз пройти по всему набору данных 

        for epoch in range(epochs):
            for x, y_true in zip(data, all_y_trues):
                # --- Прямой проход (эти значения нам понадобятся позже)
                sum_h1 = self.w1 * x[0] + self.w2 * x[1] + self.b1
                h1 = sigmoid(sum_h1)

                sum_h2 = self.w3 * x[0] + self.w4 * x[1] + self.b2
                h2 = sigmoid(sum_h2)

                sum_o1 = self.w5 * h1 + self.w6 * h2 + self.b3
                o1 = sigmoid(sum_o1)
                y_pred = o1

                # --- Считаем частные производные.
                # --- Имена: d_L_d_w1 = "частная производная L по w1"
                d_L_d_ypred = -2 * (y_true - y_pred)

                # Нейрон o1
                d_ypred_d_w5 = h1 * deriv_sigmoid(sum_o1)
                d_ypred_d_w6 = h2 * deriv_sigmoid(sum_o1)
                d_ypred_d_b3 = deriv_sigmoid(sum_o1)

                d_ypred_d_h1 = self.w5 * deriv_sigmoid(sum_o1)
                d_ypred_d_h2 = self.w6 * deriv_sigmoid(sum_o1)

                # Нейрон h1
                d_h1_d_w1 = x[0] * deriv_sigmoid(sum_h1)
                d_h1_d_w2 = x[1] * deriv_sigmoid(sum_h1)
                d_h1_d_b1 = deriv_sigmoid(sum_h1)

                # Нейрон h2
                d_h2_d_w3 = x[0] * deriv_sigmoid(sum_h2)
                d_h2_d_w4 = x[1] * deriv_sigmoid(sum_h2)
                d_h2_d_b2 = deriv_sigmoid(sum_h2)

                # --- Обновляем веса и пороги
                # Нейрон h1
                self.w1 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w1
                self.w2 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w2
                self.b1 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_b1

                # Нейрон h2
                self.w3 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w3
                self.w4 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w4
                self.b2 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_b2

                # Нейрон o1
                self.w5 -= learn_rate * d_L_d_ypred * d_ypred_d_w5
                self.w6 -= learn_rate * d_L_d_ypred * d_ypred_d_w6
                self.b3 -= learn_rate * d_L_d_ypred * d_ypred_d_b3

              # --- Считаем полные потери в конце каждой эпохи
            if epoch % 10 == 0:
                y_preds = np.apply_along_axis(self.feedforward, 1, data)
                loss = mse_loss(all_y_trues, y_preds)
                print("Epoch %d loss: %.3f" % (epoch, loss))

# Определим набор данных
data = np.array([
  [-2, -1],  # Алиса
  [25, 6],   # Боб
  [17, 4],   # Чарли
  [-15, -6], # Диана
])
all_y_trues = np.array([
  1, # Алиса
  0, # Боб
  0, # Чарли
  1, # Диана
])

# Обучаем нашу нейронную сеть!
network = OurNeuralNetwork()
network.train(data, all_y_trues)

Потестируем на ранее не виденных данных:

In [ ]:
emily = np.array([-7, -3]) # 128 фунтов (52.35 кг), 63 дюйма (160 см)
frank = np.array([20, 2])  # 155 фунтов (63.4 кг), 68 дюймов (173 см)
print(f"Эмили: {network.feedforward(emily):.3f}")
print(f"Фрэнк: {network.feedforward(frank):.3f}")